In [1]:
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import os

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"
savePath = "s3://corpus-2/model/freenli/model"

In [3]:
def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])

        for i in range(num):
            np.random.shuffle(train_data[i])
            train_data[i] = np.asarray(train_data[i])

        np.random.shuffle(train_data)   
        return train_data
    
    
    train_data = []
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, split_type):
    
    print(split_type)
    
    train_acc = []
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(tx[test_i],ty[test_i])
            train_acc.append(model_train(train_x, train_y,split_type,fold_id))
            test_acc.append(model_evaluate(test_x, test_y,split_type,fold_id,categories))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_acc.append(model_train(tx[train_i], ty[train_i],split_type,fold_id))
            test_acc.append(model_evaluate(tx[test_i], ty[test_i],split_type,fold_id,categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
class ZhouCLSTMModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1511.08630
    '''
    def __init__(self, embedding,
        conv_size    = 3,
        conv_filters = 300,
        drop_rate    = 0.5,
        lstm_units   = 150):
        '''Constructor.
        # Parameters:
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        conv_filters: Number of convolution filters.
        drop_rate: Drop rate for the final output of the LSTM layer.
        lstm_units: Size of the states of the LSTM layer.
        '''
        self._embedding    = embedding
        self._conv_size    = conv_size
        self._conv_filters = conv_filters
        self._drop_rate    = drop_rate
        self._lstm_units   = lstm_units

    def __call__(self, x_input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, x_input)
        
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size,
            self._conv_filters,
            self._drop_rate,
            self._embedding_tf)
        
        self._lstm_tf = self._create_lstm_layer(
            self._lstm_units,
            self._convolution_tf)
        
        return self._lstm_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('lstm:', str(self._lstm_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self,
        conv_size, num_filters, drop_rate, embedding):
        
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, 1, num_filters]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[num_filters]))

        embedding_expanded = tf.expand_dims(embedding, -1)
        conv = tf.nn.conv2d(
            input=embedding_expanded,
            filter=W,
            strides=[1,1,1,1],
            padding='VALID')
        conv_reduced = tf.reshape(
            tensor=conv,
            shape=[-1, conv.shape[1], conv.shape[3]])

        bias = tf.nn.bias_add(conv_reduced, b)
        c = tf.nn.relu(bias)

        d = tf.nn.dropout(c, keep_prob=drop_rate)
        return d

    def _create_lstm_layer(self, lstm_units, conv_input):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_units)
        sequence = tf.unstack(conv_input, axis=1)
        (_, (h, _)) = tf.nn.static_rnn(lstm_cell, sequence, dtype=tf.float32)

        return h

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

In [6]:
seq_length = 41

In [7]:
vocab, embd = loadGloVe(vocabPath, 100)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
# embedding = np.asarray(embd)
embedding = embd

Loading GloVe!
Completed!


In [8]:
word_to_id = dict(zip(vocab, range(vocab_size)))
len(embedding),embedding_dim,vocab_size

(400000, 100, 400000)

In [9]:
def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [9]:
learning_rate = 1e-3
dropout_keep_prob = 0.5

# 输入内容及对应的标签
input_x = tf.placeholder(tf.int32, [None, seq_length], name='input_x')
input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

model = ZhouCLSTMModel(embedding, drop_rate = keep_prob)
fc = model(input_x)
model.summary()

# 分类器
logits = tf.layers.dense(fc, num_classes, name='fc2')
y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）

# 损失函数，交叉熵
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=input_y)
loss = tf.reduce_mean(cross_entropy)
# 优化器
optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 准确率
correct_pred = tf.equal(tf.argmax(input_y, 1), y_pred_cls)
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


saver = tf.train.Saver()

embedding: (?, 41, 100)
conv: (?, 39, 300)
lstm: (?, 150)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [12]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {input_x: x_batch1, input_y: y_batch1, keep_prob: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len

In [13]:
train_data = dataset_split(split_info["random"])

In [19]:
len(train_data), len(train_data[0])

(14035, 2)

In [16]:
x_train, y_train = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)

In [17]:
len(x_train)

14035

In [18]:
# 创建session
session = tf.Session()
session.run(tf.global_variables_initializer())

print('Training and evaluating...')
start_time = time.time()
total_batch = 0  # 总批次
best_acc_train = 0.0  # 最佳验证集准确率
last_improved = 0  # 记录上一次提升批次
num_epochs = 50
batch_size = 64
require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
print_per_batch = 30  # 每多少轮输出一次结果
flag = False

for epoch in range(num_epochs):  # 20
    print('Epoch:', epoch + 1)
    batch_train = batch_iter(x_train, y_train, batch_size)
    for x_batch, y_batch in batch_train:
        feed_dict = {input_x: x_batch, input_y: y_batch, keep_prob: dropout_keep_prob}
        session.run(optim, feed_dict=feed_dict)  # 运行优化
        total_batch += 1

        if total_batch % print_per_batch == 0:
            # 每多少轮次输出在训练集和验证集上的性能
            feed_dict[keep_prob] = 1.0
            loss_train, acc_train = session.run([loss, acc], feed_dict=feed_dict)
            # loss_val, acc_val = evaluate(session, x_dev, y_dev, loss, acc)
            if acc_train > best_acc_train:
                # 保存最好结果
                best_acc_train = acc_train
                last_improved = total_batch
                saver.save(sess=session, save_path=savePath)
                improved_str = '*'
            else:
                improved_str = ''

            time_dif = get_time_dif(start_time)
            msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%}, Time: {3} {4}'
            print(msg.format(total_batch, loss_train, acc_train, time_dif, improved_str))

        if total_batch - last_improved > require_improvement:
            # 验证集正确率长期不提升，提前结束训练
            print("No optimization for a long time, auto-stopping...")
            flag = True
            break  # 跳出循环
    if flag:  # 同上
        break

session.close()

Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.0, Train Acc:  43.75%, Time: 0:00:14 *
Iter:     60, Train Loss:    1.7, Train Acc:  40.62%, Time: 0:00:17 
Iter:     90, Train Loss:    1.4, Train Acc:  46.88%, Time: 0:00:29 *
Iter:    120, Train Loss:    1.3, Train Acc:  51.56%, Time: 0:00:44 *
Iter:    150, Train Loss:   0.77, Train Acc:  73.44%, Time: 0:00:54 *
Iter:    180, Train Loss:   0.79, Train Acc:  70.31%, Time: 0:00:58 
Iter:    210, Train Loss:   0.74, Train Acc:  73.44%, Time: 0:01:02 
Epoch: 2
Iter:    240, Train Loss:   0.36, Train Acc:  87.50%, Time: 0:01:13 *
Iter:    270, Train Loss:   0.57, Train Acc:  81.25%, Time: 0:01:17 
Iter:    300, Train Loss:   0.35, Train Acc:  85.94%, Time: 0:01:20 
Iter:    330, Train Loss:    0.4, Train Acc:  89.06%, Time: 0:01:36 *
Iter:    360, Train Loss:   0.31, Train Acc:  89.06%, Time: 0:01:39 
Iter:    390, Train Loss:   0.33, Train Acc:  89.06%, Time: 0:01:43 
Iter:    420, Train Loss:   0.47, Train Acc:  84.38%

In [20]:
def preprocess_sentence(sent):
    new_sent = ''
    for i in range(len(sent)):
        if sent[i] in string.punctuation:
            if i > 0 and i < len(sent) - 1:
                if sent[i] in ",." and sent[i-1].isdigit() and sent[i+1].isdigit():
                    new_sent += sent[i]
                    continue
                if sent[i] == "%" and sent[i-1].isdigit():
                    new_sent += sent[i]
                    continue
                if sent[i] == "$" and (sent[i-1].isdigit() or sent[i+1].isdigit()):
                    new_sent += sent[i]
                    continue
                if sent[i-1] != ' ':
                    new_sent += ' ' + sent[i]
                elif sent[i+1] != ' ':
                    new_sent += sent[i] + ' '
                else:
                    new_sent += sent[i]
            elif i == 0:
                if sent[i] == "$" and sent[i+1].isdigit():
                    new_sent += sent[i]
                    continue
                if sent[i+1] != ' ':
                    new_sent += sent[i] + ' '
                else:
                    new_sent += sent[i]
            else:
                if sent[i] == "%" and sent[i-1].isdigit():
                    new_sent += sent[i]
                    continue
                if sent[i] == "$" and sent[i-1].isdigit():
                    new_sent += sent[i]
                    continue
                if sent[i-1] != ' ':
                    new_sent += ' ' + sent[i]
                else:
                    new_sent += sent[i]
        else:
            new_sent += sent[i]
    return new_sent.strip().lower()


def predict11(predict_sentences, probability_threshold=0.3):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    data_id = []
    # 将文本内容转换为对应的id形式
    for psi in predict_sentences:

        data_id.append([word_to_id[x] for x in preprocess_sentence(psi).split() if x in word_to_id])

    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, seq_length)
    feed_dict = {
        input_x: x_pad,
        keep_prob: 1.0
    }
    predict_result = session.run(tf.nn.softmax(logits), feed_dict=feed_dict)
    print(predict_result)
    result = []
    for i in predict_result:
        if max(i) > probability_threshold:
            result.append(i.argmax()+1)
        else:
            result.append(0)
    return result

In [22]:
import string

In [24]:
session = tf.Session()
saver.restore(sess=session, save_path=savePath)

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/freenli/model


INFO:tensorflow:Restoring parameters from s3://corpus-2/model/freenli/model


In [40]:
sentences = ["can you tell me what is arkansas 's population on the date july 1st of 2002 ?",
"show the way the number of likes were distributed .", "is it true that people living on average depends on higher gdp of a country"]  # 1,7,10

predict11(sentences)

[[9.9565226e-01 3.8410417e-05 3.1449313e-03 7.6289487e-04 3.6055729e-04
  9.4450970e-06 1.7177252e-05 5.7368757e-06 1.2711318e-07 8.3821969e-06]
 [4.2775994e-07 2.0520065e-06 6.1646038e-05 4.1338234e-07 1.9917552e-07
  3.5080964e-05 9.9974602e-01 4.9381674e-07 1.3718965e-04 1.6582457e-05]
 [1.4377131e-05 1.9067187e-04 2.9857219e-03 6.4738197e-06 2.6175562e-06
  6.4397642e-07 2.5312585e-04 9.0113713e-04 2.2385353e-05 9.9562281e-01]]


[1, 7, 10]